## Assets and Portfolios

### Imports

In [1]:
import os

os.chdir("..")

Here are some of the relevant imports:

In [2]:
from pytrading.assets import (
    reset,
    Asset,
    Cash,
    Stock,
    FxRate,
    Portfolio,
)

### Basic Usage

In [3]:
reset()

cash = Cash("USD")            # USD cash
stock = Stock("AAA US")       # A stock with the ticker 'AAA US'
portfolio = Portfolio("USD")  # A portfolio denominated (value measured) in USD

In [4]:
print(cash)

Cash('USD', 1.0, currency_code='USD')


In [5]:
print(stock)

Stock('AAA US', None, currency_code='USD')


In [6]:
print(portfolio)

Portfolio('USD')


Note that cash always has a static local price of 1.0. We haven't initialised the price of our stock yet (so it's None).

In [7]:
cash.price, stock.price

(1.0, None)

We can set the price of our stock at any time, but we're not allowed to set the price of cash (which is static).

In [8]:
stock.price = 10

In [9]:
try:
    # cash will always have a value of $1.0. Trying to set it will throw an attribute error
    cash.price = 10
except Exception as e:
    print(e)

can't set attribute


In [10]:
cash.price, stock.price

(1.0, 10)

In [11]:
cash.local_value, stock.local_value

(1.0, 10.0)

### A note on asset codes

In [12]:
cash.code, stock.code

('USD', 'AAA US')

Codes are intended to be unique. Attempting to create an asset with a code that is already taken will throw an error.

In [13]:
try:
    stock2 = Stock("AAA US")
except Exception as e:
    print(e)

Code is already in use.


The code 'AAA US' is weakly linked to the stock object and will only be removed when the stock object is destroyed.

In [14]:
Asset.get_asset_for_code("AAA US") is stock

True

If attempting to restart a backtest and re-initialize assets then I'd suggest running the reset() function first (which will clear asset code associations).

### Seeding our portfolio

Once the assets and portfolios are created we can seed the portfolio by transferring units.

In [15]:
portfolio.transfer(cash, 1000)
portfolio.transfer(stock, 100)

In [16]:
print(portfolio)

Portfolio('USD'):
Cash('USD', 1.0, currency_code='USD'): 1,000
Stock('AAA US', 10, currency_code='USD'): 100


In [17]:
portfolio.value

2000.0

### The observable / observer relationship between assets and portfolio

Our portfolio will observe movements in the assets it holds and revalue itself accordingly.

In [18]:
stock.price = 11
print(portfolio)
print("*" * 10)
print("portfolio value is: ", portfolio.value)

Portfolio('USD'):
Cash('USD', 1.0, currency_code='USD'): 1,000
Stock('AAA US', 11, currency_code='USD'): 100
**********
portfolio value is:  2100.0


### Movements in FX rates
pytrading supports multiple currencies. For example, you could have an AUD denominated portfolio holding assets in USD, GBP, EUR... The only requirement is that FX rates must be avaliable for the portfolio to use in valuation. FxRateEvent is a supported event type which we'll cover later. For this example we'll just set the rates manually.

In [19]:
reset()

aud = Cash("AUD")
usd = Cash("USD")
stock = Stock("AAA US", currency_code="USD")
audusd = FxRate("AUDUSD")

In [20]:
audusd.pair, audusd.rate

('AUDUSD', None)

In [21]:
audusd.rate = 0.7
stock.price = 10

In [22]:
portfolio = Portfolio("AUD")

for asset, amount in [(aud, 1000), (usd, 1000), (stock, 100)]:
    portfolio.transfer(asset, amount)

print(portfolio)

Portfolio('AUD'):
Cash('AUD', 1.0, currency_code='AUD'): 1,000
Cash('USD', 1.0, currency_code='USD'): 1,000
Stock('AAA US', 10, currency_code='USD'): 100


In [23]:
portfolio.value  # value of portfolio in denominated currency ('AUD')

3857.142857142857

How is this portfolio value arrived at?
*  1000 AUD => AUD value = 1000
*  1000 USD => AUD value = 1000 / 0.7
*  100 shares => AUD value = 100 * 10 / 0.7

In [24]:
1000 + 1000 / 0.7 + 100 * 10 / 0.7

3857.142857142857

Also note that the portfolio will observe movements in FX rates that are relevant to its valuation. If, for example, the AUDUSD rate declines then the value of USD holdings (measured in AUD) will increase and the portfolio value will increase.

In [25]:
audusd.rate = 0.65
portfolio.value

4076.923076923076

### Static vs. Variable price assets

Asset is the base class for all assets. From the Asset class we can branch out into static price assets and variable price assets.

In [26]:
from pytrading.assets import (
    StaticPriceAsset,
    VariablePriceAsset,
)

In [27]:
issubclass(StaticPriceAsset, Asset), issubclass(VariablePriceAsset, Asset)

(True, True)

Assets such as cash have a static value when measured in their local currency.

In [28]:
issubclass(Cash, StaticPriceAsset)

True

Other assets such as stocks, for example, have a variable price.

In [29]:
issubclass(Stock, VariablePriceAsset)

True

Why make the distinction between static and variable priced assets? Portfolios need to observe assets: when an asset price changes the portfolio revalues itself. However, static price assets do not need to be observed as their prices do not change. However, associated FX rates for all assets will have to be observed if they are not denominated in the same currency as our portfolio.

***